In [30]:
def affine_gap_alignment(seq1, seq2, weight_match, weight_mismatch, open_gap_penalty, continue_gap_penalty):
    n = len(seq1)
    m = len(seq2)

    M_main = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    M_main[0] = [open_gap_penalty + i * continue_gap_penalty for i in range(n + 1)]

    for j in range(m + 1):
        M_main[j][0] = open_gap_penalty + j * continue_gap_penalty
    
    M_main[0][0] = 0
    
    M_low = M_main.copy()
    M_up = M_main.copy()

    for i in range(1, m + 1):  # seq2
        for j in range(1, n + 1):  # seq1        
            def seq_match(a, b):
                if a == b:
                    return weight_match
                else:
                    return weight_mismatch
            
            s1_up = M_up[i-1][j] + continue_gap_penalty
            s2_up = M_main[i-1][j] + open_gap_penalty + continue_gap_penalty
            
            M_up[i][j] = max(s1_up, s2_up)
            
            s1_low = M_low[i][j-1] + continue_gap_penalty
            s2_low = M_main[i][j-1] + open_gap_penalty + continue_gap_penalty
            
            M_low[i][j] = max(s1_low, s2_low)
            
            s1 = M_main[i-1][j-1] + seq_match(seq1[j - 1], seq2[i - 1])
            s2 = M_up[i][j]
            s3 = M_low[i][j]
            
            s = max(max(s1, s2), s3)
            M_main[i][j] = s
    
    seq_new1 = ''
    seq_new2 = ''

    cur_i = m
    cur_j = n

    while cur_i > 0 or cur_j > 0:
        if M_main[cur_i - 1][cur_j - 1] >= M_low[cur_i][cur_j - 1] and M_main[cur_i - 1][cur_j - 1] >= M_up[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
            cur_j = cur_j - 1
        elif M_low[cur_i][cur_j - 1] > M_main[cur_i - 1][cur_j - 1] and M_low[cur_i][cur_j - 1] > M_up[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = '_' + seq_new2
            cur_j = cur_j - 1
        elif M_up[cur_i - 1][cur_j] > M_main[cur_i - 1][cur_j - 1] and M_up[cur_i - 1][cur_j] > M_low[cur_i][cur_j - 1]:
            seq_new1 = '_' + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1

    print(seq_new1, seq_new2)

In [32]:
sequence1, sequence2 = 'actgagt agctact'.split()

affine_gap_alignment(sequence1, sequence2, 3, -3, -2, -1)

([[0, -3, -4, -5, -6, -7, -8, -9],
  [-3, 3, 2, 1, 0, -1, -2, -3],
  [-4, -5, 0, -1, 4, 3, 2, 1],
  [-5, -6, -2, -3, -4, 1, 0, -1],
  [-6, -7, -8, 1, 0, -1, -2, 3],
  [-7, -3, -4, -5, -2, 3, 2, 1],
  [-8, -9, 0, -1, -2, -3, 0, -1],
  [-9, -10, -11, 3, 2, 1, 0, 3]],
 [[0, -3, -4, -5, -6, -7, -8, -9],
  [-3, 3, 2, 1, 0, -1, -2, -3],
  [-4, -5, 0, -1, 4, 3, 2, 1],
  [-5, -6, -2, -3, -4, 1, 0, -1],
  [-6, -7, -8, 1, 0, -1, -2, 3],
  [-7, -3, -4, -5, -2, 3, 2, 1],
  [-8, -9, 0, -1, -2, -3, 0, -1],
  [-9, -10, -11, 3, 2, 1, 0, 3]],
 [[0, -3, -4, -5, -6, -7, -8, -9],
  [-3, 3, 2, 1, 0, -1, -2, -3],
  [-4, -5, 0, -1, 4, 3, 2, 1],
  [-5, -6, -2, -3, -4, 1, 0, -1],
  [-6, -7, -8, 1, 0, -1, -2, 3],
  [-7, -3, -4, -5, -2, 3, 2, 1],
  [-8, -9, 0, -1, -2, -3, 0, -1],
  [-9, -10, -11, 3, 2, 1, 0, 3]])

In [27]:
MM[1][0] = 22
MM

[[0, -3, -4, -5, -6, -7, -8],
 [22, 0, 0, 0, 0, 0, 0],
 [-4, 0, 0, 0, 0, 0, 0],
 [-5, 0, 0, 0, 0, 0, 0],
 [-6, 0, 0, 0, 0, 0, 0],
 [-7, 0, 0, 0, 0, 0, 0],
 [-8, 0, 0, 0, 0, 0, 0],
 [-9, 0, 0, 0, 0, 0, 0]]

In [ ]:
[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [ ]:
ind = {'a': 0,
            't': 1,
            'g': 2,
            'c': 3,
            '_': 4,}

# weight matrix

W = [[ 1, -1, -1, -1, -1],
     [-1,  1, -1, -1, -1],
     [-1, -1,  1, -1, -1],
     [-1, -1, -1,  1, -1],
     [-1, -1, -1, -1,  1],]

In [ ]:
# global alignment for comparison

def global_alignment(seq1, seq2):
    n = len(seq1)
    m = len(seq2)

    M = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    M[0] = [-i for i in range(n + 1)]

    for j in range(m + 1):
        M[j][0] = -j

    for i in range(1, m + 1):  # seq2
        for j in range(1, n + 1):  # seq1        

            s1 = M[i-1][j-1] + W[ind[seq2[i - 1]]][ind[seq1[j - 1]]]
            s2 = M[i-1][j] + W[ind[seq2[i - 1]]][ind['_']]
            s3 = M[i][j-1] + W[ind['_']][ind[seq1[j - 1]]]

            s = max(max(s1, s2), s3)
            M[i][j] = s

    seq_new1 = ''
    seq_new2 = ''

    cur_i = m
    cur_j = n

    while cur_i > 0 or cur_j > 0:
        if M[cur_i - 1][cur_j - 1] >= M[cur_i][cur_j - 1] and M[cur_i - 1][cur_j - 1] >= M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1
            cur_j = cur_j - 1
        elif M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j - 1] and M[cur_i][cur_j - 1] > M[cur_i - 1][cur_j]:
            seq_new1 = seq1[cur_j - 1] + seq_new1
            seq_new2 = '_' + seq_new2
            cur_j = cur_j - 1
        elif M[cur_i - 1][cur_j] > M[cur_i - 1][cur_j - 1] and M[cur_i - 1][cur_j] > M[cur_i][cur_j - 1]:
            seq_new1 = '_' + seq_new1
            seq_new2 = seq2[cur_i - 1] + seq_new2
            cur_i = cur_i - 1

    print(seq_new1, seq_new2, sep='\n')